In [23]:
import numpy as np
from names_dataset import NameDataset
from graphviz import Digraph
from utils import draw_dot

In [4]:
namedataset = NameDataset()
many_names = namedataset.get_top_names(500, use_first_names=True, country_alpha2='IT', gender='F')['IT']['F']

In [5]:
print(len(many_names))
print(type(many_names))
print(many_names[-20:])

500
<class 'list'>
['Daiana', 'Mariantonietta', 'Nicol', 'Bella', 'Carol', 'Magdalena', 'Madalina', 'Luminita', 'Mila', 'Cornelia', 'Chicca', 'Grace', 'Olivia', 'Priscilla', 'Myriam', 'Samuela', 'Cri', 'Maria Carmela', 'Annunziata', 'Marilisa']


In [6]:
unique_chars = set()
for name in many_names:
    unique_chars.update(set(name.lower()))
    if ' ' in name:
        print(name)
        many_names.remove(name)
        unique_chars.remove(' ')


unique_chars = sorted(list(unique_chars))
print(len(many_names))
print(len(unique_chars))
print(unique_chars)



Anna Maria
Maria Grazia
Maria Teresa
Maria Rosaria
Maria Luisa
Maria Rosa
Maria Antonietta
Maria Cristina
Maria Rita
Maria Concetta
Maria Giovanna
Maria Chiara
Maria Elena
Maria Vittoria
Ana Maria
Maria Angela
Rosa Maria
Maria Carmela
482
26
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [7]:
S_to_I = {s:i for i,s in enumerate(unique_chars)}
I_to_S = {i:s for i,s in enumerate(unique_chars)}

print(S_to_I)
print(I_to_S)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}


z: Data = 15.0, Grad = 1.0
y: Data = 5.0, Grad = 0
x: Data = 2.0, Grad = 0
